In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np

In [2]:
#Select mnist dataset
mnist = tf.keras.datasets.mnist

#Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#And convert to float
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
#Create neural network
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [8]:
#Test untrained model
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.29659358, -1.2483128 , -0.01258914, -0.9083958 , -0.09044996,
        -0.3696527 ,  0.6248741 , -0.6964865 , -0.16109064,  0.08964382]],
      dtype=float32)

In [9]:
#Convert test output to probability
tf.nn.softmax(predictions).numpy()

array([[0.1504984 , 0.0321061 , 0.11047285, 0.04510365, 0.10219768,
        0.07730095, 0.20897885, 0.05574973, 0.09522746, 0.12236431]],
      dtype=float32)

In [4]:
#Initialize loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
#Calculate loss of test output
loss_fn(y_train[:1], predictions).numpy()

2.560049

In [5]:
#Compile model for training
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [6]:
#Train model
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2978 - accuracy: 0.9136
Epoch 2/5
60000/60000 [==============================] - 9s 142us/sample - loss: 0.1481 - accuracy: 0.9554
Epoch 3/5
60000/60000 [==============================] - 9s 142us/sample - loss: 0.1081 - accuracy: 0.9672
Epoch 4/5
60000/60000 [==============================] - 9s 143us/sample - loss: 0.0884 - accuracy: 0.9723
Epoch 5/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.0757 - accuracy: 0.9764


In [7]:
#Evaluate model on new data
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 1s - loss: 0.0745 - accuracy: 0.9778


[0.07448177339471877, 0.9778]

In [8]:
#Add probability function to create new model with human output
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
#Show random image from test data

(_, _), (x_test, y_test) = mnist.load_data()

i = np.random.randint(0, len(x_test))

Image.fromarray(x_test[i]).show() #x_test must be int so this line works

a = np.ndarray(shape=(1,28,28))
a[0] = x_test[i]

probability_model(a).numpy()

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [13]:
#Test on self made image

im = Image.open("test.png").convert("L")

a = np.ndarray(shape=(1,28,28))
a[0] = np.asarray(im)# if you add /255 here it shows percentages

probability_model(a).numpy()

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)